# 1. Introduction:

## 1.1 Research Topic

Stereochemistry is an important concept in organic chemistry for understanding molecules' behavior in biological systems. Indeed, the spatial arrangement of atoms in a molecule can significantly influence how that molecule interacts with enzymes and other biomolecules. Stereoisomers—compounds that have the same molecular formula but differ in the 3D orientation of their atoms—often exhibit different properties. A thorough understanding of stereochemistry is therefore essential in fields such as pharmaceutical chemistry, where the difference between isomers can mean the difference between a therapeutic and a toxic effect.

## 1.2 Settings: Niche

While there are tools for drawing molecules and naming stereoisomers, few are designed to be interactive learning tools. Most tools and materials explain stereochemistry in a serious and technical way, but they are not fun or easy to use. Because of that, students often find it hard to really understand the topic. Furthermore, current tools often do not simplify the challenge of identifying stereocenters or visualizing all possible stereoisomers for a given molecule, especially from linear notations like SMILES.

## 1.3 Problem

Students often struggle when it comes to understanding and visualizing stereoisomers, particularly when translating 2D representations into 3D structures. As a result, students may fail to understand how chirality and molecular orientation influence chemical behavior. After struggling with the concepts of stereochemistry, the idea of this project seemed obvious to us: we wanted to create an interface to get acquainted with stereoisomers, nomenclature, and chirality. Thus, in this project, we tried to put as many functionalities as possible into a Streamlit interface, which we hope will serve as a helpful and intuitive tool for users.

## 1.4 Solution

This project presents a Python-based tool that automatically detects chiral centers and calculates the number of possible stereoisomers of a given molecule. Using cheminformatics libraries such as RDKit, the tool processes SMILES input and outputs an intuitive stereochemical analysis. The aim is to create an accessible, educational resource that not only aids students in mastering stereochemistry but also serves as a basis for further computational chemistry applications.

# 2. Materials and methods


This application was developed using Python and Streamlit to create an interactive tool for exploring and studying stereochemistry, with a particular focus on stereoisomer identification and naming. The tool integrates molecule drawing, stereoisomer generation, and chiral center identification into a user-friendly interface.

## 2.1 Technologies and libraries

The primary technologies used in the development of this project include:

- **Python 3.8**: The main programming language.
- **Streamlit**: Used for building the web interface and handling user interactions.
- **RDKit**: Employed for molecule parsing, stereoisomer generation, chiral center identification, and image rendering.
- **PubChemPy**: Utilized for retrieving IUPAC names from the PubChem database.
- **streamlit_ketcher**: Integrated to allow users to draw chemical structures directly in the application interface.

All dependencies were installed via pip and the application was run locally through the Streamlit CLI.

## 2.2 Streamlit application structure

The application is organized into three main tabs:

1. **Input a Molecule**:
    Users can either draw a molecule using the Ketcher editor or enter a molecule name which is then interpreted using PubChemPy. For both input methods, the application converts the structure into a canonical SMILES representation and removes any stereochemistry, ensuring that isomer enumeration starts from a consistent base.

2. **Draw and Guess Stereoisomers**:
    The core functionality of this tab involves the generation of all possible stereoisomers using a custom function, `generate_isomers`, which relies on RDKit’s stereoisomer enumeration capabilities. Users attempt to draw valid stereoisomers of the input molecule, which are then checked for correctness by comparing their canonical isomeric SMILES to the generated set. The application provides feedback, tracks user guesses, and includes features such as scorekeeping, hints, and timing mechanisms. Users can also validate their drawn isomers by inputting corresponding IUPAC names, which are compared against PubChem-derived names.
    A score system is also established, assigning a point for all correct answers. 

3. **Chirality**:
    In this section, users can try to identify the chiral centers of the input molecule. The RDKit library is used to determine the atoms with possible chirality, which are then visually highlighted in molecular images. Users select atoms they believe to be chiral using checkboxes.

A sidebar is also present on the left side of the application. Here, it is possible to input a molecule by name, and the current chosen molecule (without sterical information) is displayed using RDKit for easier visualization during the stereoisomer guessing. 

## 2.3 Variables management

The application relies on Streamlit's session state system to store variables during its usage. This mechanism allows the app to remember variables between user interactions. Without this method, information would be lost because Streamlit re-runs the entire script after any change. Therefore, session state is used to keep track of the user's progress and decisions throughout the application; the session state variables are only updated or deleted when explicitely instructed to do so. 

Variables stored in session state includes, among others, the input molecule selected by the user, the stereoisomers they have guessed so far, their current score, any names they have submitted for validation, and whether they have requested a hint or chosen to reveal the correct answers. 

By storing this data in session state, the application can provide a seamless and responsive experience where progress is preserved and the interface remains interactive.

## 2.4 User experience

The interface includes a custom background image for visual appeal and uses Streamlit’s layout capabilities (such as columns and placeholders) to dynamically update feedback messages, images, and scores. Correct answers are rewarded with balloons as a playful visual effect. 

## 2.5 Implementation details

This section describes the main ideas used to implement the functionality in the "Draw isomers" and "Chirality" tabs, followed by an overview of two central functions that support the application's internal logic while keeping the overall code cleaner. 

### Guessing the isomers (Tab 2)
In this tab, users are asked to guess all the possible isomers of the molecule that has previously input. The programming logic begins by generating the full set of stereoisomers from the SMILES string of the input molecule. These stereoisomers are stored as a set of canonical SMILES strings that retain stereochemical detail (i.e., using @ for R/S centers or \ and / for E/Z bonds). This set is treated as the reference for comparison with the isomers drawn by the user.

When users enter their guesses through the ketcher interface, the retrieved SMILES string is canonicalized and checked by comparing it with the reference set. If the guess matches a stereoisomer that is present in the set of "solutions" and that had not been guessed yet, it is recorded as correct and added to the set of correct isomers that have been found. This structure enables real-time feedback without storing redundant data, and avoids false positives due to duplicate or equivalent representations.

The second part of the tab challenges users to name the specific stereoisomers that they have drawn using correct IUPAC nomenclature. Users must correctly describe its configuration (R/S or E/Z) in their answer. The program validates this by comparing the input name with the name retrieved from the molecule's SMILES using PubChem.

### Chirality (Tab 3)
This tab asks the user to identify all the chiral centers of the input molecule. When a molecule (as a SMILES string) is available in the session, it converts the SMILES into an RDKit molecule object. It then iterates through each atom, checking the RDKit `_ChiralityPossible` property to detect potential chiral centers, and stores their indices in a list. 

The program then compares the user’s selection through checkboxes (which is also a list) with the actual chiral atom list (chiral_atoms). If they match, a success message and animation are triggered, controlled via a session state flag (balloons_shown). Finally, optional buttons allow users to toggle the display of the correct chiral atoms, storing the visibility preference in st.session_state.show_chiral_atoms.


### Helper functions
Two helper functions support much of the core functionality described above: `generate_isomers` and `update_input_molecule`.

The `generate_isomers` function takes a SMILES string and returns a set of all unique stereoisomers corresponding to that molecule. It relies on RDKit’s built-in enumeration tools to identify all undefined stereocenters and systematically generate the missing stereochemical variants. Importantly, it uses enumeration options that ignore already-defined centers, so only ambiguous or incomplete stereochemistry is expanded. The output is returned as a set of canonical SMILES strings with full stereochemical specification, which are then used for validation, comparison, and selection throughout the app.

The `update_input_molecule` function is used to reset the application state each time a new molecule is selected.
It updates the main SMILES string and clears all relevant session variables, such as previously guessed isomers, score, answer visibility, validated names, and timers. This ensures the user starts fresh with every new challenge.


# 3. Results

# 4. Discussion

# 5. References
